In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
from gptchem.data import get_moosavi_mof_data
from sklearn.model_selection import train_test_split
from gptchem.baselines.henry import FEATURES
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier

import pandas as pd

from pycm import ConfusionMatrix

In [2]:
data = get_moosavi_mof_data()

In [9]:
data['logKH_CH4_bins'] = pd.qcut(data['logKH_CH4'], 5, labels=[0, 1, 2, 3, 4])

In [14]:
train_data, test_data = train_test_split(data, train_size=100)

In [15]:
len(train_data)

100

In [16]:
X_train, y_train = train_data[FEATURES], train_data["logKH_CH4_bins"]
X_test, y_test = test_data[FEATURES], test_data["logKH_CH4_bins"]

In [17]:
selector = SelectFromModel(estimator=RandomForestClassifier(), max_features=100)
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)
tabpfn = TabPFNClassifier(device="cpu", N_ensemble_configurations=32)

tabpfn.fit(X_train, y_train)

tabpfn_predictions, _ = tabpfn.predict(X_test, return_winning_probability=True)

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


In [18]:
tabpfn_predictions

array([1, 2, 4, ..., 0, 2, 2])

In [19]:
y_test

5170    2
1052    0
1862    3
2807    0
807     4
       ..
106     4
2231    0
5664    0
2430    2
5143    1
Name: logKH_CH4_bins, Length: 6090, dtype: category
Categories (5, int64): [0 < 1 < 2 < 3 < 4]

In [24]:
cm = ConfusionMatrix(y_test.values.astype(int), tabpfn_predictions)

In [25]:
print(cm)

Predict   0         1         2         3         4         
Actual
0         763       187       178       7         80        

1         330       344       444       0         100       

2         206       246       548       10        208       

3         132       171       532       21        366       

4         172       97        300       17        631       





Overall Statistics : 

95% CI                                                            (0.36663,0.391)
ACC Macro                                                         0.75153
ARI                                                               0.09206
AUNP                                                              0.61183
AUNU                                                              0.61197
Bangdiwala B                                                      0.18881
Bennett S                                                         0.22352
CBA                                                               0.3